# NN Pipeline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import json
from datetime import datetime
import random

from sklearn.preprocessing import PowerTransformer, StandardScaler, RobustScaler, MinMaxScaler


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization,Dropout, SimpleRNN
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanAbsolutePercentageError
from tensorflow.keras import losses
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten


from sklearn.model_selection import train_test_split



from scipy.special import boxcox1p, inv_boxcox1p
from scipy import stats
from scipy.stats import norm, skew

from sklearn.preprocessing import PolynomialFeatures


import warnings
warnings.filterwarnings("ignore")


%matplotlib inline

pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 100)

sns.set_style("white")
matplotlib.rc('xtick', labelsize=15)
matplotlib.rc('ytick', labelsize=15)
plt.rcParams['figure.figsize'] = [16.0, 10.0]

def mean_absolute_percentage_error(y_pred, y_true):
    y_true = np.where(y_true == 0, 0.0000000001, y_true)
    return np.mean(np.abs((y_true - y_pred) / y_true))

# Preprocesamiento

In [ ]:
class DataFramePreProcessor:
    
    def __init__(self, dataframe, test=False):
        self.test = test
        self.original_dataframe = dataframe.copy()
        self.modeling_dataframe = None

    
    
    def handleMissingData(self, dataframe):
        dataframe['ingreso_final'] = dataframe['ingreso_final'].fillna(0)
        dataframe['ind'] = dataframe['ind'].fillna(0)
        dataframe['tipo_vivienda'] = dataframe['tipo_vivienda'].fillna("NO INFORMA")
        dataframe['categoria'] = dataframe['categoria'].fillna("6")
        dataframe['estado_civil'] =dataframe['estado_civil'].fillna("NI")
        #dataframe['departamento_residencia'] = dataframe['departamento_residencia'].fillna("SIN INFORMACION")
        #dataframe['ocupacion'] = dataframe['ocupacion'].fillna("Otro")
        dataframe['ind_mora_vigente']  = dataframe['ind_mora_vigente'].fillna("N") 
        return dataframe
    
    
    def columnFilter(self, dataframe):
        to_drop = [
            "Unnamed: 0",
            "id_cli",
            "pol_centr_ext",
            "ult_actual",
            "cant_mora_30_tdc_ult_3m_sf",
            "cant_mora_30_consum_ult_3m_sf",
            #"ind",
            "cat_ingreso",
            "departamento_residencia",
            "ocupacion",
            "estado_civil",
            "tipo_vivienda",
            "nivel_academico",
            "rep_calif_cred",
            "tiene_ctas_activas",
            "mora_max"
            
        ]
        
            
        dataframe = dataframe.drop(to_drop,axis=1, errors='ignore')
        return dataframe
    
    
    # Borrar filas deacuerdo a cierta logica de negocio
    def rowFilter(self, dataframe):
        cut = 0.999999
        return  dataframe[
                (dataframe['gasto_familiar'] >= 50000) &
                (dataframe['gasto_familiar'] < np.quantile(dataframe['gasto_familiar'], cut)) &
                (dataframe['ingreso_final'] < np.quantile(dataframe['ingreso_final'], cut)) &
                (dataframe['cupo_total_tc'] < np.quantile(dataframe['cupo_total_tc'], cut)) & # Percentil 99%
                (dataframe['cuota_tc_bancolombia'] < np.quantile(dataframe['cuota_tc_bancolombia'], cut)) & # percentil 99.99%
                (dataframe['cuota_de_vivienda'] < np.quantile(dataframe['cuota_de_vivienda'], cut)) &# Percentil 99.99%
                (dataframe['cuota_de_consumo'] < np.quantile(dataframe['cuota_de_consumo'], cut)) & # percentil 99%
                (dataframe['cuota_rotativos'] < np.quantile(dataframe['cuota_rotativos'], cut))& # percentil 99.99%
                (dataframe['cuota_tarjeta_de_credito'] < np.quantile(dataframe['cuota_tarjeta_de_credito'], cut)) & 
                (dataframe['cuota_de_sector_solidario'] < np.quantile(dataframe['cuota_de_sector_solidario'], cut)) &
                (dataframe['cuota_sector_real_comercio'] < np.quantile(dataframe['cuota_sector_real_comercio'], cut)) &# Percentil 99.5%
                (dataframe['cuota_libranza_sf'] < np.quantile(dataframe['cuota_libranza_sf'], cut)) & # Percentil 99
                (dataframe['ingreso_segurida_social'] < np.quantile(dataframe['ingreso_segurida_social'], cut)) & # percentil 99.9
                (dataframe['ingreso_nomina'] < np.quantile(dataframe['ingreso_nomina'], cut)) &
                (dataframe['saldo_prom3_tdc_mdo'] < np.quantile(dataframe['saldo_prom3_tdc_mdo'], cut)) &
                (dataframe['saldo_no_rot_mdo'] < np.quantile(dataframe['saldo_no_rot_mdo'], cut)) &
                (dataframe['cuota_cred_hipot'] < np.quantile(dataframe['cuota_cred_hipot'], cut)) &
                (dataframe['mediana_nom3'] < np.quantile(dataframe['mediana_nom3'], cut)) &
                (dataframe['mediana_pen3'] < np.quantile(dataframe['mediana_pen3'], cut)) &
                (dataframe['cuota_tc_mdo'] < np.quantile(dataframe['cuota_tc_mdo'], cut)) &
                (dataframe['ingreso_nompen'] < np.quantile(dataframe['ingreso_nompen'], cut)) &
                (dataframe['cant_oblig_tot_sf'] <= 15) &
                (dataframe['ctas_activas'] < 5) &
                (dataframe['nro_tot_cuentas'] < 5) &
                (dataframe['ind_mora_vigente'] != 'NApl') &
                (dataframe['ind'] < np.quantile(dataframe['ind'], cut))
              #  ~(dataframe['departamento_residencia'].isin(['MADRID', 'ESTADO DE LA FLORIDA', 'VAUPES']))
            ] 
    
    def oneEncodeVariables(self):
        pass
    
    def processVars(self, dataframe):
        
        # Transformacion demograficas
        dataframe['genero'] = np.where(dataframe['genero'] == 'M', 0, 1)
        #dataframe['edad'] = np.where(dataframe['edad'] < 18, 18,
        #                            np.where(dataframe['edad'] > 80, 80, dataframe['edad']
        #                            ))
       # dataframe['educacion_grupo'] = np.where(
       #     dataframe['nivel_academico'].isin(['PRIMARIO', 'UNIVERSITARIO', 'ESPECIALIZACION']),1,0
       #)
        #dataframe['ind_caida_gasto'] = np.where(dataframe['periodo'] == 202004,1,0)
        #dataframe['ind_caida_cuota'] = np.where(dataframe['periodo'] == 202009,1,0)
        #dataframe['tipo_vivienda'] =  np.where(dataframe['tipo_vivienda'].isin(["\\N", "NO INFORMA"]),"OTRO",
        #                                       np.where(dataframe['tipo_vivienda'].isin(['FAMILIAR','ALQUILADA']), "FAM_ALQ",
        #                                                dataframe['tipo_vivienda']
        #                                               )
        #                                     )
        dataframe['categoria']=np.where(dataframe['categoria']=='\\N', "6",dataframe['categoria'])
        dataframe['categoria']=dataframe['categoria'].astype(float).astype(int)
        #dataframe['ocupacion']=np.where(dataframe['ocupacion'].isin(
        #        ["\\N", "Agricultor", "Ganadero", 'Vacío', "Ama de casa", "Sin Ocupacion Asignada"]), "Otro",
        #                               np.where(dataframe['ocupacion'] == "Pensionado", "Jubilado", dataframe['ocupacion']
        #                                       ))
        
        #dataframe['departamento_residencia'] = \
        #    np.where(
        #        dataframe['departamento_residencia'].isin(['\\N', 'NARIÑO', 'NARI#O', 'VAUPES','MADRID', 'ESTADO DE LA FLORIDA']),
        #        "SIN INFORMACION", dataframe['departamento_residencia'] )
        
        #dataframe['es_ciudad_principal'] = np.where(
        #    dataframe['departamento_residencia'].isin(['BOGOTA D.C.', 'ANTIOQUIA', 'VALLE', 'CUNDINAMARCA']), 1,0)  
        #dataframe['estado_civil'] = np.where(dataframe['estado_civil'].isin(['NI', 'VIU', 'OTRO', 'NI', 'DIV']), 
        #                                     "OTRO", dataframe['estado_civil'])
        
        #dataframe['ind_mora_vigente'] = np.where(dataframe['ind_mora_vigente'] == "S", 1, 0)
        dataframe['convenio_lib'] = np.where(dataframe['convenio_lib'] == 'S', 1, 0)
        # Transformacion finacieras
        dataframe['ingreso_calculado'] =  dataframe['ingreso_segurida_social']  +  \
                                          dataframe[['ingreso_nompen', 'ingreso_nomina']].max(axis=1)
        dataframe['ingreso_corr'] = dataframe[['ingreso_final', 'ingreso_calculado']].max(axis=1)
        
        dataframe['cuota_cred_hipot'] = dataframe[['cuota_cred_hipot', 'cuota_de_vivienda']].max(axis=1)
        dataframe['cuota_de_consumo'] =np.where(dataframe['cuota_de_consumo'] < 0, 0, dataframe['cuota_de_consumo'])
        dataframe['cuota_cred_hipot'] =np.where(dataframe['cuota_cred_hipot'] < 0, 0, dataframe['cuota_cred_hipot']) 
        
        dataframe['total_cuota'] = dataframe['cuota_cred_hipot'] + \
                                   dataframe['cuota_tarjeta_de_credito'] + \
                                   dataframe['cuota_de_consumo'] + \
                                   dataframe['cuota_rotativos'] + \
                                   dataframe['cuota_de_sector_solidario'] + \
                                   dataframe['cuota_libranza_sf'] + \
                                   dataframe['cuota_tarjeta_de_credito'] + \
                                   dataframe['cuota_tc_bancolombia']
        
        #dataframe['saldo_favor']  = np.where(dataframe['total_cuota']<0, dataframe['total_cuota']*-1, 0)
        dataframe['total_cuota'] = np.where(dataframe['total_cuota']<0,0, dataframe['total_cuota'])

        #dataframe['cat_edad'] = np.where(dataframe['edad'] < 30, "M30",
        #                          np.where(dataframe['edad'] < 40, "M30_40",
        #                                  np.where(dataframe['edad'] < 50, "M40_50", "M60")))
                
        pct_vars = [
            'cuota_cred_hipot',
            'cuota_tarjeta_de_credito',
            'cuota_de_consumo',
            'cuota_rotativos',
            'cuota_sector_real_comercio',
            'cuota_de_sector_solidario',
            'cuota_tc_bancolombia',
            'cuota_libranza_sf',
            'cupo_total_tc',
            'cupo_tc_mdo'
        ]
        
        for var in pct_vars:
            dataframe[f"{var}_pct"] = dataframe[var] / dataframe['ingreso_corr']  * 100
            dataframe[f"{var}_pct"] = dataframe[f"{var}_pct"].replace(dict.fromkeys([np.nan, np.inf, -np.inf], 0))
            #dataframe[f"{var}_2"]   = dataframe[var] ** 2
    
        dataframe['obl_total_pct'] = dataframe['cuota_cred_hipot_pct'] + \
                                     dataframe['cuota_tarjeta_de_credito_pct'] +\
                                     dataframe['cuota_de_consumo_pct'] + \
                                     dataframe['cuota_rotativos_pct'] + \
                                     dataframe['cuota_sector_real_comercio_pct'] + \
                                     dataframe['cuota_de_sector_solidario_pct'] + \
                                     dataframe['cuota_tc_bancolombia_pct'] + \
                                     dataframe['cuota_rotativos_pct'] 
        
                                    

        #dataframe['sobre_endeudado'] = np.where(dataframe['obl_total_pct'] > 100, 1, 0)
        dataframe['total_cupo'] = dataframe['cupo_total_tc'] + dataframe['cupo_tc_mdo']
        #dataframe['ingreso_cero'] = np.where(dataframe['ingreso_corr'] == 0, 1, 0)
        
        # Variables de interaccion
        #dataframe['interact_ing_gen']  = dataframe['genero'] * dataframe['ingreso_corr']
        #dataframe['interact_ing_ed']  = dataframe['edad'] * dataframe['ingreso_corr']
        #dataframe['interact_cup_gen']  = dataframe['genero'] * dataframe['total_cupo']
        #dataframe['interact_cup_ed']  = dataframe['edad'] * dataframe['total_cupo']
        #dataframe['interact_obl_gen'] = dataframe['genero'] * dataframe['obl_total_pct']
        #dataframe['interact_obl_ed']  = dataframe['edad'] * dataframe['obl_total_pct']    
        #dataframe['interact_caida_ing'] = dataframe['ingreso_corr']*dataframe['ind_caida_gasto']
        #dataframe['interact_mora_ing'] = dataframe['ingreso_corr']*dataframe['ind_mora_vigente']
        #dataframe['interact_mora_cuota'] = dataframe['total_cuota']*dataframe['ind_mora_vigente']
        #dataframe['interact_caida_cuota'] = dataframe['ind_caida_cuota'] * dataframe['total_cuota']
        
        #dataframe['ingreso_geo_alto']  = np.where(dataframe['ingreso_corr'] < 14.90, 1, 0) # ALgo mas tecnico

    
        #dataframe['pc25'] = np.where(dataframe['ingreso_corr'] <= np.quantile(dataframe['ingreso_corr'],0.25), 1, 0)
        #dataframe['pc75'] = np.where(dataframe['ingreso_corr'] >= np.quantile(dataframe['ingreso_corr'],0.75), 1, 0)

        # variables al cuadrado
        
        #dataframe['edad_2'] = dataframe['edad']**2
        dataframe['total_cuota_2']  =dataframe['total_cuota']**2
        #dataframe['total_cupo_2']  =dataframe['total_cupo']**2
        #dataframe['obl_total_pct_2'] = dataframe['obl_total_pct']**2
        #dataframe['ingreso_corr2'] = dataframe['ingreso_corr']**2 
        
        
        # Raiz de las variables
        
        #dataframe['cupo_pct'] = dataframe['total_cupo']/dataframe['ingreso_corr']
        #dataframe['cupo_disponible'] = dataframe['total_cupo'] - dataframe['cuota_tarjeta_de_credito'] - \
         #                              dataframe['cuota_tc_bancolombia']
        #dataframe['liquidez'] = dataframe['cupo_disponible'] + dataframe['ingreso_corr']
        #dataframe['liquidez_c'] = dataframe['total_cupo'] + dataframe['ingreso_corr']
        #dataframe['cuota_pct_cupo'] = (dataframe['cuota_tarjeta_de_credito'] + dataframe['cuota_tc_bancolombia']) / dataframe['total_cupo']
        #dataframe['ind_corregido'] = dataframe['ingreso_corr'] - dataframe['total_cuota'] - dataframe['ingreso_corr']*0.1
        #dataframe['ratio_cupo'] = dataframe['cupo_tc_mdo']/dataframe['cupo_tc_mdo']
        
        dataframe.replace([np.inf, -np.inf], np.nan, inplace=True)
        dataframe.fillna(0, inplace=True)
        dataframe['periodo'] = dataframe['periodo'].astype(str)
        #dataframe = dataframe.merge(indicators.drop(['inflacion'], axis=1), left_on='periodo', right_on='Fecha')
        
        if not self.test:
            #dataframe['gasto_familiar'] = np.where(dataframe['gasto_familiar'] <= 0,
            #                                       1,
            #                                      dataframe['gasto_familiar'])
            
            dataframe['log_gasto_familiar'] = np.log1p(dataframe['gasto_familiar']) 
            #dataframe['log_gasto_familiar'] = np.sqrt(dataframe['gasto_familiar'])
            numeric_feats = dataframe.drop(['gasto_familiar', 'log_gasto_familiar'], axis=1).dtypes[
                    (dataframe.dtypes == "float64")].index
        else:
            numeric_feats = dataframe.dtypes[
                    (dataframe.dtypes == "float64")].index
            
        skewed_feats = dataframe[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
        skewness = pd.DataFrame({'Skew' :skewed_feats})
        skewness = skewness[abs(skewness) > 0.75]       
        skewed_features = skewness.index
        lam = 0.1
        PT_transformer = PowerTransformer()
        for feat in skewed_features:
             #dataframe[feat] = boxcox1p(dataframe[feat], lam)        
            dataframe[feat] = PT_transformer.fit_transform(dataframe[feat].values.reshape(-1,1))
        
            
        cat_vars = [
            #'mora_max',
            #'estado_civil',
            #'rep_calif_cred',
            #'ocupacion',
            'tipo_vivienda',
            #'nivel_academico',
            #'departamento_residencia',
            #"ind_mora_vigente",
            #"cat_edad",
            #"periodo"
        ]
        if cat_vars:
            dummified = []
            for var in cat_vars:            
                dummified.append(
                    pd.get_dummies(dataframe[var], drop_first=True, prefix=var)
                )

            dummified = pd.concat(dummified, axis=1)
            dataframe = pd.concat([dataframe.drop(cat_vars, axis=1),dummified], axis=1)
            
        cluster_variables = [
            'cuota_cred_hipot',
            'cuota_tarjeta_de_credito',
            'cuota_de_consumo',
            'cuota_rotativos',
            'cuota_sector_real_comercio',
            'cuota_de_sector_solidario',
            'cuota_tc_bancolombia',
            'cuota_libranza_sf',
            "ingreso_corr",
            "cupo_tc_mdo",
            "nro_tot_cuentas",
            "mediana_nom3",
            "mediana_pen3",
            "cant_oblig_tot_sf",
            "saldo_prom3_tdc_mdo",
            "cupo_total_tc",
            "saldo_no_rot_mdo"
        ]
            
        #gmm =GaussianMixture(n_components=5, random_state=1).fit_predict(dataframe[cluster_variables])
        #dataframe['cluster'] = gmm
        ## Final cleaning
        
        dataframe.drop(["ingreso_final", "ingreso_calculado",
                        "cuota_de_vivienda", 
                        "gasto_familiar",
                        "cupo_total_tc_pct", "cupo_tc_mdo_pct", "cupo_disponible",
                        #"cupo_tc_mdo",
                        #"cupo_total_tc",
                        "ingreso_nompen", "ingreso_nomina", "ingreso_segurida_social",
                        "ingreso_cero",
                        "tenencia_tc", # Alta correlacion con cuota tarjeta de credito
                        "ctas_activas", # Alta correlacion con nto_tot_cuentas
                        "cuota_cred_hipot_pct", #altta correlacion con cuota credito hipotecario
                        "cuota_de_consumo_pct", #Alta correlacion con cuta de consumo
                        "cuota_rotaticos_pct", # Alta correlacion con cuota de rotativos
                        "cuota_tarjeta_de_credito_pct",
                        "cuota_de_sector_solidario_pct",
                        "cuota_sector_real_comercio_pct",
                        "cuota_libranza_sf_pct",
                        "cuota_tc_bancolombia_pct",
                        "cuota_rotativos_pct",
                        # REVISAR BIEN
                        #"saldo_no_rot_mdo",
                        "total_cuota",
                        "convenio_lib",
                        "ind_mora_vigente",
                        #'cuota_cred_hipot',
                        #'cuota_tarjeta_de_credito',
                        #'cuota_de_consumo',
                        #'cuota_rotativos',
                        #'cuota_sector_real_comercio',
                        #'cuota_de_sector_solidario',
                        #'cuota_tc_bancolombia',
                        #'cuota_libranza_sf',
                        "tiene_consumo",
                        #"cuota_tc_mdo",
                        "tiene_crediagil",
                        #"nivel_academico",
                        "total_cupo",
                        #"genero",
                        #"nro_tot_cuentas",
                        #"mediana_nom3",
                        #"mediana_pen3",
                        #"cant_oblig_tot_sf",
                        #"saldo_prom3_tdc_mdo",
                        #"categoria",
                        "cuota_pct_cupo",
                        "edad",
                        #"ind_caida_gasto",
                        #"ind_caida_cuota",
                        #"Fecha",
                        #"periodo"
                        "mora_max",
                       ], axis=1, inplace=True,  errors='ignore')

        return dataframe
    
    def process(self):
        
        complete_df = self.columnFilter(
                self.handleMissingData(self.original_dataframe)
        )
        if not self.test:
            filtered_df = self.rowFilter(complete_df)
            grown_df    = self.processVars(filtered_df)
        else:
            grown_df    = self.processVars(complete_df)
        self.modeling_dataframe = grown_df
        
        return self.modeling_dataframe
        

# Ensamble

In [ ]:
print(f"Hora de inicio: {datetime.now()}")
dates = [
    '201902', '201903', '201904', '201905', '201907', '201908', '201909',
    '201910', '201911', '202001', '202002', '202003', '202004', '202005',
    '202007', '202008', '202009', '202010', '202011'
]

test_df = pd.read_csv("test_cleaned.csv")
test_df['cuota_de_consumo'] = np.where(test_df['cuota_de_consumo'] <0 ,0, test_df['cuota_de_consumo'])
test_df_modeling = DataFramePreProcessor(test_df, test=True)
test_df_modeling.process()

last_predictions_list = []
y_tests = []
x_tests = []
y_preds = []
mapes   = []
models = []
p = 0.001

callback = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
optimizer   = Adam(learning_rate=0.003)
nn_metric = MeanAbsolutePercentageError(name='mape') 

random.seed(123)

with open('tuned_hyper_parameters.json') as f:
    tuned_hyperparameters = json.load(f)

for date in dates:
    print(f"Periodo {date}:")
    raw_dataframe = pd.read_csv(
             f"train_{date}_cleaned.csv",
             header=0,
             skiprows=lambda i: i>0 and random.random() > p
    ).drop("Unnamed: 0", axis=1)
    
    print(f"     Total rows in original_data {raw_dataframe.shape[0]}" )
    fe_dataframe = DataFramePreProcessor(raw_dataframe)
    fe_dataframe.process()
    X = fe_dataframe.modeling_dataframe.drop([ "log_gasto_familiar",
                                         "periodo"], axis=1)
    y = fe_dataframe.modeling_dataframe['log_gasto_familiar']
    print(f"     Total rows in transformed_data {fe_dataframe.modeling_dataframe.shape[0]}" )
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=111)

    
    # RED NEURONAL
    nn_model = Sequential()
    
    nn_model.add(Dense(128, activation = 'relu', input_shape = (X.shape[1],))) 
    nn_model.add(Dropout(0.5))
    nn_model.add(BatchNormalization())
    nn_model.add(Dense(64, activation = 'tanh'))
    nn_model.add(Dropout(0.5))
    nn_model.add(Dense(1))
    nn_model.compile(optimizer =  optimizer, loss = 'mse', metrics = nn_metric)
    history  = nn_model.fit(X_train, y_train, epochs=20, callbacks=[callback], validation_split = 0.2)
    
    
    models.append(nn_model)

    nn_pred = nn_model.predict(X_test).reshape(-1,)
    y_tests.append(y_test)
    x_tests.append(X_test)

    mape = mean_absolute_percentage_error(nn_pred, y_test)
    mapes.append(mape)
    print(f"     MAPE {date}: ", mape )

print(f"Hora de finalizacion: {datetime.now()}")

# Predicciones

In [ ]:
predictions_df = test_df_modeling.modeling_dataframe.drop(["id_registro", "periodo"], axis=1)

nn_stack_predictions = []
for model in models:
    nn_stack_predictions.append(model.predict(predictions_df).reshape(-1,))
nn_stack = np.expm1(np.mean(nn_stack_predictions, axis=0))

In [ ]:
submission = pd.concat([test_df['id_registro'],pd.Series(nn_stack)], axis=1)
submission.columns = ["id_registro", "gasto_familiar"]
submission['gasto_familiar'] = submission['gasto_familiar'].round(4)

In [ ]:
submission.to_csv("submission_underground_nn.csv", index=False)